In [17]:
import sys
import os

folder_path = os.path.abspath("/projappl/project_2013104/pengyan1/venv/lib/python3.10/site-packages")
if folder_path not in sys.path:
    sys.path.append(folder_path)

sys.path

['C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\\python312.zip',
 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\\DLLs',
 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\\Lib',
 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0',
 '',
 'C:\\Users\\yanpe\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages',
 'C:\\Users\\yanpe\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\win32',
 'C:\\Users\\yanpe\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\win32\\lib',
 'C:\\Users\\yanpe\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Pyth

In [48]:
import numpy as np
from torch.utils.data import random_split
import torch
import torch.nn as nn
import loralib as lora
import torch.nn.functional as F
from torchmetrics import JaccardIndex
import torch_geometric.transforms as T
from torch_geometric.nn import MLP
from torch_geometric.loader import DataLoader
from torch_geometric.typing import WITH_TORCH_CLUSTER

from pyg_pointnet2 import PyGPointNet2NoColor
from pc_dataset import H5PCDataset

if not WITH_TORCH_CLUSTER:
    quit("This example requires 'torch-cluster'")

In [70]:
# Empty the CUDA cache
torch.cuda.empty_cache()

In [50]:
# take out colors
class SelectLast3Features:
    def __call__(self, data):
        # If data.x is defined, select only its last 3 features.
        if data.x is not None:
            data.x = data.x[:, -3:]
        return data

# transform and pre_transform
transform = T.Compose([
    T.RandomJitter(0.01),
    T.RandomRotate(15, axis=0),
    T.RandomRotate(15, axis=1),
    T.RandomRotate(15, axis=2),
    SelectLast3Features()
])

h5_file_path = "../docs/sim_pc_dataset.h5" # local file path
#h5_file_path ='/scratch/project_2013104/datasets/sim_pc_dataset.h5' # csc file path

full_dataset = H5PCDataset(file_path=h5_file_path, transform=transform)

# Define split sizes (e.g., 80% training and 20% validation)
total_size = len(full_dataset)
train_size = int(0.8 * total_size)
test_size = total_size - train_size

# Randomly split the dataset
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

In [51]:
print(train_dataset[0])
print(train_dataset.dataset.num_classes)

Data(x=[4096, 3], y=[4096], pos=[4096, 3])
13


In [52]:
batch_size=32
num_workers=0

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                          num_workers=num_workers, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                         num_workers=num_workers, pin_memory=True)

In [63]:
# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = PyGPointNet2NoColor(num_classes=13).to(device)

model_file_path = "checkpoints/pointnet2_s3dis_colorless_seg_x3_45_checkpoint.pth"
# Load the checkpoint dictionary
checkpoint = torch.load(model_file_path, map_location=device)
# Extract the model state dictionary
model_state_dict = checkpoint['model_state_dict']

model.load_state_dict(model_state_dict, strict=True)  

# Replace the final classification layer
#new_num_classes = 13
#model.lin3 = torch.nn.Linear(128, new_num_classes)
#model.mlp = MLP([128, 128, 128, new_num_classes], dropout=0.5, norm=None)

#model = model.to(device)

<All keys matched successfully>

In [65]:
model.eval()

PyGPointNet2NoColor(
  (sa1_module): SAModule(
    (conv): PointNetConv(local_nn=MLP(6, 64, 64, 128), global_nn=None)
  )
  (sa2_module): SAModule(
    (conv): PointNetConv(local_nn=MLP(131, 128, 128, 256), global_nn=None)
  )
  (sa3_module): GlobalSAModule(
    (nn): MLP(259, 256, 512, 1024)
  )
  (fp3_module): FPModule(
    (nn): MLP(1280, 256, 256)
  )
  (fp2_module): FPModule(
    (nn): MLP(384, 256, 128)
  )
  (fp1_module): FPModule(
    (nn): MLP(131, 128, 128, 128)
  )
  (mlp): MLP(128, 128, 128, 13)
  (lin1): Linear(in_features=128, out_features=128, bias=True)
  (lin2): Linear(in_features=128, out_features=128, bias=True)
  (lin3): Linear(in_features=128, out_features=13, bias=True)
)

In [66]:
optimizer = torch.optim.Adam(
    model.parameters(),  # All parameters are trainable
    lr=1e-4,
    weight_decay=0.01
)

In [56]:
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    total_loss = correct_nodes = total_nodes = 0
    for i, data in enumerate(train_loader):
        data = data.to(device)        
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out.view(-1, 13), data.y.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        correct_nodes += out.argmax(dim=1).eq(data.y).sum().item()
        total_nodes += data.num_nodes

        if (i + 1) % 10 == 0:
            print(f'[{i+1}/{len(train_loader)}] Loss: {total_loss / 10:.4f} '
                  f'Train Acc: {correct_nodes / total_nodes:.4f}')
            total_loss = correct_nodes = total_nodes = 0
    # If there are remaining batches that were not printed (i.e., i+1 not divisible by 10)
    if total_nodes > 0:
        num_remaining = (i + 1) % 10  # Number of batches in the leftover segment
        print(f'[{i+1}/{len(train_loader)}] Loss: {total_loss / num_remaining:.4f} '
              f'Train Acc: {correct_nodes / total_nodes:.4f}')

In [57]:

@torch.no_grad()
def test(loader):
    model.eval()
    jaccard = JaccardIndex(num_classes=loader.dataset.dataset.num_classes, task="multiclass").to(device)
    
    for data in loader:
        data = data.to(device)
        outs = model(data)
        preds = outs.argmax(dim=-1)
        jaccard.update(preds, data.y)
    
    return jaccard.compute().item()


In [67]:
# Train
import time
begin_time = time.perf_counter()
for epoch in range(1, 501):
    start_time = time.perf_counter()
    train()
    iou = test(test_loader)
    epoch_time = time.perf_counter() - start_time    
    print(f'Epoch: {epoch:02d}, Test IoU: {iou:.4f}, Time: {epoch_time:.2f}s')
total_time = time.perf_counter() - begin_time
print(f'Training time: {total_time/60:.2f}m')


[10/13] Loss: 2.2360 Train Acc: 0.6340
[13/13] Loss: 1.3020 Train Acc: 0.7130
Epoch: 01, Test IoU: 0.1900, Time: 3.72s
[10/13] Loss: 1.4314 Train Acc: 0.7012
[13/13] Loss: 1.2537 Train Acc: 0.7148
Epoch: 02, Test IoU: 0.1965, Time: 3.50s
[10/13] Loss: 1.2552 Train Acc: 0.7281
[13/13] Loss: 1.8357 Train Acc: 0.8126
Epoch: 03, Test IoU: 0.1935, Time: 3.39s
[10/13] Loss: 1.0519 Train Acc: 0.7480
[13/13] Loss: 1.5296 Train Acc: 0.7600
Epoch: 04, Test IoU: 0.2055, Time: 3.40s
[10/13] Loss: 0.8765 Train Acc: 0.7764
[13/13] Loss: 1.0275 Train Acc: 0.7919
Epoch: 05, Test IoU: 0.1979, Time: 3.43s
[10/13] Loss: 0.7986 Train Acc: 0.7953
[13/13] Loss: 1.1893 Train Acc: 0.7621
Epoch: 06, Test IoU: 0.2030, Time: 3.41s
[10/13] Loss: 0.7236 Train Acc: 0.7985
[13/13] Loss: 1.8157 Train Acc: 0.7461
Epoch: 07, Test IoU: 0.1932, Time: 3.41s
[10/13] Loss: 0.7333 Train Acc: 0.8024
[13/13] Loss: 1.2307 Train Acc: 0.7799
Epoch: 08, Test IoU: 0.1964, Time: 3.44s
[10/13] Loss: 0.7406 Train Acc: 0.7963
[13/13] L

In [68]:
torch.save(lora.lora_state_dict(model), "checkpoints/smartlab_fine_tuning_x3_500_20250415.pth")

In [69]:
del model